# Deep Agent - Multi-Agent Deep Finance Researcher

Advanced financial research system using LangChain's DeepAgent with context isolation and strategic delegation.

## Overview

This notebook demonstrates:
- **DeepAgent Framework**: LangChain's advanced agent architecture
- **Context Isolation**: Sub-agents with isolated contexts for focused research
- **Strategic Delegation**: Orchestrator coordinates multiple specialist sub-agents
- **RAG + Live Data**: Hybrid search on SEC filings + Yahoo Finance integration
- **File-Based Memory**: Persistent research artifacts with secure sandbox (virtual_mode=True)

**Key Features:**
1. Orchestrator breaks down complex queries into research tasks
2. Sub-agents conduct focused research with isolated context
3. Think tool for strategic reflection and decision-making
4. Final report synthesis with consolidated citations
5. SQLite checkpointer for conversation memory
6. FilesystemBackend with virtual_mode for secure file operations

## Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
import sqlite3
from datetime import datetime

from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.sqlite import SqliteSaver

from scripts.rag_tools import hybrid_search, live_finance_researcher, think_tool

from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend

from scripts.deep_prompts import DEEP_RESEARCHER_INSTRUCTIONS, DEEP_ORCHESTRATOR_INSTRUCTIONS

## Initialize Memory and File Backend

In [ ]:
# Secure filesystem backend for research outputs
RESEARCH_OUTPUT_DIR = os.path.join(os.getcwd(),"research_outputs")

In [ ]:
def get_research_backend(user_id, thread_id):

    USER_OUTPUT_DIR = os.path.join(RESEARCH_OUTPUT_DIR, user_id, thread_id)
    
    os.makedirs(USER_OUTPUT_DIR, exist_ok=True)

    print(f"Writing research files to: {USER_OUTPUT_DIR}")

    # Create filesystem backend with virtual_mode=True for security
    backend = FilesystemBackend(
        root_dir=USER_OUTPUT_DIR,
        virtual_mode=True  # Prevents agent from accessing files outside sandbox
    )

    return backend

## Create Research Sub-Agent

In [ ]:
# Get current date
current_date = datetime.now().strftime("%Y-%m-%d")

# Create research sub-agent with isolated context
research_sub_agent = {
    "name": "financial-research-agent",
    "description": "Delegate financial research to this sub-agent. Give it one specific research task at a time.",
    "system_prompt": DEEP_RESEARCHER_INSTRUCTIONS.format(date=current_date),
    "tools": [hybrid_search, live_finance_researcher, think_tool],
}

## Initialize LLM and Create DeepAgent

In [ ]:
# Initialize model
model = ChatGoogleGenerativeAI(model='gemini-3-flash-preview')

# Tools for the main agent (orchestrator level)
tools = [hybrid_search, live_finance_researcher, think_tool]

In [ ]:
def get_deep_agent(user_id, thread_id):

    # SQLite checkpointer for agent memory
    conn = sqlite3.connect('data/deep_agent_finance_researcher.db', check_same_thread=False)
    checkpointer = SqliteSaver(conn=conn)

    backend = get_research_backend(user_id, thread_id)

    # Create the deep agent with memory and secure file backend
    agent = create_deep_agent(
        model=model,
        tools=tools,
        system_prompt=DEEP_ORCHESTRATOR_INSTRUCTIONS,
        subagents=[research_sub_agent],
        checkpointer=checkpointer,  # SQLite memory
        backend=backend,  # Secure filesystem with virtual_mode=True
    )

    return agent

    # Visualize the agent graph
    agent

## Examples

In [ ]:
from scripts.agent_utils import stream_agent_response

query = "What was Amazon's revenue in Q1 2024?"
user_id = "kgptalkie"
thread_id="session1"

agent = get_deep_agent(user_id, thread_id)
stream_agent_response(agent, query, thread_id)

In [ ]:
query = "Compare Apple and Amazon's 2024 revenue and profitability. Present full and detailed report."
thread_id="session2"

agent = get_deep_agent(user_id, thread_id)
stream_agent_response(agent, query, thread_id)

In [ ]:
query = "Research Meta's 2024 annual financial performance from SEC filings and compare it with current stock performance"
thread_id="session4"

agent = get_deep_agent(thread_id)
stream_agent_response(agent, query, thread_id)